In [178]:
import re
import srt 
import glob
import pandas as pd
import os
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kirill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kirill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kirill\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [133]:
files = glob.glob("./English_level/data_all/Subtitles_all/*.srt")

In [134]:
HTML = r'<.*?>'
TAG = r'{.*?}'
COMMENTS = r'[\(\[][A-Z ]+[\)\]]'
LETTERS = r'[^a-zA-Z\'.,!? ]'
SPACES = r'([ ])\1+'
DOTS = r'[\.]+'

def clean_subs(subs):
    txt = re.sub(HTML, ' ', subs) #html тэги меняем на пробел
    txt = re.sub(TAG, ' ', txt) #тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt) #комменты меняем на пробел
    txt = re.sub(LETTERS, ' ', txt) #все что не буквы меняем на пробел
    txt = re.sub(SPACES, r'\1', txt) #повторяющиеся пробелы меняем на один пробел
    txt = re.sub(DOTS, r'.', txt)  #многоточие меняем на точку
    # txt = txt.encode('ascii', 'ignore').decode() #удаляем все что не ascii символы   
    # txt = ".".join(txt.lower().split('.')[1:-1]) #удаляем первый и последний субтитр (обычно это реклама)
    return txt

In [135]:
dframes = {}

for filename in files:
    try:
        with open(filename) as f:
            subtitle_generator = srt.parse(f.read().encode('ascii', 'ignore').decode())
            df = pd.DataFrame(list([[item.start, item.end, item.content] for item in subtitle_generator]), columns=['start','end','content'])
            df['content'] = df['content'].apply(clean_subs)
            df['content'] = df['content'].str.strip().replace('', np.nan).str.lower()
            df.dropna(subset=['content'], inplace=True)
            df.reset_index(drop=True, inplace=True)

            dframes[os.path.basename(f.name)] = df

    except srt.SRTParseError as e:
        print(filename, e)

In [136]:
YEAR_SRT = r'(\(\d{4}\))'

In [137]:
names_df = pd.DataFrame([n for n in dframes], columns=['name'])
names_df['year'] = names_df['name'].str.extract(YEAR_SRT, flags=0)
names_df['parsed_name'] = names_df.apply(lambda x: x['name'].replace(str(x['year']), '').replace('_', ' ').replace('.srt', ''), axis=1)

In [138]:
names_df

,name,year,parsed_name
0,10_Cloverfield_lane(2016).srt,(2016),10 Cloverfield lane
1,10_things_I_hate_about_you(1999).srt,(1999),10 things I hate about you
2,Aladdin(1992).srt,(1992),Aladdin
3,All_dogs_go_to_heaven(1989).srt,(1989),All dogs go to heaven
4,An_American_tail(1986).srt,(1986),An American tail
...,...,...,...
109,Warm_bodies(2013).srt,(2013),Warm bodies
110,Westworld_scenes_of_Dr_Robert_Ford.srt,NaN,Westworld scenes of Dr Robert Ford
111,We_are_the_Millers(2013).srt,(2013),We are the Millers
112,While_You_Were_Sleeping(1995).srt,(1995),While You Were Sleeping


In [139]:
labels = pd.read_csv('English_level/data_all/labels_all.csv')
display(labels)

,Movie,Kinopoisk,Level,Subtitles
0,Forrest Gump,Rus sub,"A2/A2+, B1",Yes
1,Finding Nemo\n,Everything,A2/A2+,Yes
2,Cast away\n,"Paid, Rus sub",A2/A2+,Yes
3,The invisible man (2020)\n,"Paid, Rus lan",A2/A2+,Yes
4,Back to the future\n,Rus sub,A2/A2+,Yes
...,...,...,...,...
111,Klaus,NaN,C1,NaN
112,Ocean’s Eleven,NaN,C1,NaN
113,Ocean’s Twelve,NaN,C1,NaN
114,Bridget Jones’s Baby,NaN,C1,NaN


In [140]:
for index in labels.index:
    item = labels.loc[index, 'Movie']
    found = process.extractOne(item, names_df['parsed_name'])
    labels.loc[index, ['found', 'probability', 'index']] = found

labels['index'] = labels['index'].astype(int)
    

In [141]:
names_df.join(labels.set_index('index'))

,name,year,parsed_name,Movie,Kinopoisk,Level,Subtitles,found,probability
0,10_Cloverfield_lane(2016).srt,(2016),10 Cloverfield lane,10 Cloverfield Lane,NaN,B1,Yes,10 Cloverfield lane,100.0
1,10_things_I_hate_about_you(1999).srt,(1999),10 things I hate about you,10 things I hate about you,No subs,B1,Yes,10 things I hate about you,100.0
2,Aladdin(1992).srt,(1992),Aladdin,Aladdin,Everything,A2/A2+,Yes,Aladdin,100.0
3,All_dogs_go_to_heaven(1989).srt,(1989),All dogs go to heaven,All dogs go to heaven,Nope,A2/A2+,Yes,All dogs go to heaven,100.0
4,An_American_tail(1986).srt,(1986),An American tail,An American tail,Nope,A2/A2+,Yes,An American tail,97.0
...,...,...,...,...,...,...,...,...,...
109,Warm_bodies(2013).srt,(2013),Warm bodies,Warm bodies,"Everything, Paid",B1,Yes,Warm bodies,100.0
110,Westworld_scenes_of_Dr_Robert_Ford.srt,NaN,Westworld scenes of Dr Robert Ford,NaN,NaN,NaN,NaN,NaN,NaN
111,We_are_the_Millers(2013).srt,(2013),We are the Millers,We’re the Millers,Nope,B1,Yes,We are the Millers,97.0
112,While_You_Were_Sleeping(1995).srt,(1995),While You Were Sleeping,While You Were Sleeping,NaN,B1,NaN,While You Were Sleeping,100.0


In [190]:
text = '|'.join(dframes['10_Cloverfield_lane(2016).srt']['content'])

In [202]:
lemmatizer = WordNetLemmatizer()
# lemmatizer = Mystem()

stop_words = stopwords.words('english') 

In [204]:
def del_stopwords(text):
    clean_tokens = tuple(
        map( lambda x: x if x not in stop_words else '', word_tokenize(text) )
    )
    res_text = ' '.join(clean_tokens)
    return res_text
    
def lemmatize(text):    
    lemmatized_text = ''.join(m.lemmatize(text))
    return lemmatized_text.split('|')

In [205]:
lemmatize("i can't believe you just left")

["i can't believe you just left\n"]

In [198]:
lemmatize(del_stopwords(text))

['fixed synced  bozxphd . enjoy  flick',
 "ben  phone michelle , please  n't hang up.",
 "just talk   , okay ?  ca n't believe   left.",
 'michelle.',
 'come back.',
 'please say something.',
 'michelle , talk  me.',
 'look ,    argument . couples fight.',
 'that   reason   leave everything behind.',
 "running away  n't gon na help   . michelle , please.",
 'newscaster  details  that.',
 'elsewhere today , power  still   restored',
 'to many cities   southern seaboard',
 "in  wake   afternoon 's widespread blackout.",
 'while     inclement weather   region , ',
 'the problem seems linked   authorities  calling',
 'a catastrophic power surge   crippled traffic   area.',
 'no.',
 'no ! ',
 'damn.',
 'okay . okay , please.',
 'please.',
 "please  n't hurt me.",
 'please.',
 "just let  go , okay ?  wo n't tell anybody.",
 'i promise , okay ? please  let  go . please.',
 'man  need fluids .    shock.',
 'what   going     ? ',
 "i 'm going  keep  alive.",
 'work  getting handy  these.',
 'my

In [196]:
text.split('|')

['fixed synced by bozxphd. enjoy the flick',
 "ben on phone michelle, please don't hang up.",
 "just talk to me, okay? i can't believe you just left.",
 'michelle.',
 'come back.',
 'please say something.',
 'michelle, talk to me.',
 'look, we had an argument. couples fight.',
 'that is no reason to just leave everything behind.',
 "running away isn't gonna help it any. michelle, please.",
 'newscaster more details on that.',
 'elsewhere today, power has still not been restored',
 'to many cities on the southern seaboard',
 "in the wake of this afternoon's widespread blackout.",
 'while there had been some inclement weather in the region,',
 'the problem seems linked to what authorities are calling',
 'a catastrophic power surge that has crippled traffic in the area.',
 'no.',
 'no!',
 'damn.',
 'okay. okay, please.',
 'please.',
 "please don't hurt me.",
 'please.',
 "just let me go, okay? i won't tell anybody.",
 'i promise, okay? please just let me go. please.',
 'man you need fluid

In [163]:

  
''.join(lemmatizer.lemmatize(text))#.split('|')

"fixed synced by bozxphd. enjoy the flick|ben on phone michelle, please don't hang up.|just talk to me, okay? i can't believe you just left.|michelle.|come back.|please say something.|michelle, talk to me.|look, we had an argument. couples fight.|that is no reason to just leave everything behind.|running away isn't gonna help it any. michelle, please.|newscaster more details on that.|elsewhere today, power has still not been restored|to many cities on the southern seaboard|in the wake of this afternoon's widespread blackout.|while there had been some inclement weather in the region,|the problem seems linked to what authorities are calling|a catastrophic power surge that has crippled traffic in the area.|no.|no!|damn.|okay. okay, please.|please.|please don't hurt me.|please.|just let me go, okay? i won't tell anybody.|i promise, okay? please just let me go. please.|man you need fluids. you were in shock.|what are you going to do to me?|i'm going to keep you alive.|work on getting handy 